In [1]:
#STEP 1 - import packages
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import plotly as plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
plotly.tools.set_credentials_file(username='martijnvanvlie', api_key='nDD1QdIOq3rOEO51bazE')

#set specific color
pblue = 'rgb(31,119,180)'
porange = 'rgb(255,127,14)'

In [2]:
#STEP 2 - Load in data
df_traffic = pd.read_csv('Traffic_segment_data.csv', encoding='utf-8',sep = ",")
df_BMSS_traffic_link = pd.read_csv('Bridge_data_with_link_to_traffic.csv',encoding='utf-8',sep = ",")

#calculate segment score (we weigth vulnerability and criticality equally)
df_traffic["segment_score"] = ""
df_traffic["Road_cat"] = ""
df_traffic["segment_score"] = df_traffic["segment_vulnerability_normalized"] * df_traffic["Economic_Traffic_normalized"]

#get type of road to plot in scatter
df_traffic["Road_cat"] =  df_traffic['Road_Name'].astype(str).str[0]

#get top-10 segments of all segments of all roads
df_traffic = df_traffic.sort_values(by='segment_score',ascending = False)
top_points_df = df_traffic[0:10]
top_points_df["numeric"] = ""

for row in range(len(top_points_df)):
    if top_points_df.Road_cat.iloc[row,] == "N":
        top_points_df.numeric.iloc[row,] = 1
    if top_points_df.Road_cat.iloc[row,] == "R":
        top_points_df.numeric.iloc[row,] = 2

#interactive transport scatter plot data
df = pd.read_csv('Traffic_segment_data.csv')
df.drop(df.columns[0:1], axis=1,inplace = True)
df.drop('segment_vulnerability_length', axis = 1, inplace = True)
df.drop('missing_segment', axis = 1, inplace = True)
df=df.rename(columns = {'segment_vulnerability_normalized':'Vulnerability'})
df=df.rename(columns = {'Economic_Traffic_normalized':'Criticality'})
df.Criticality = df.Criticality.apply(pd.to_numeric)
df.dropna(inplace=True)

In [3]:
#Step 3 - Interactive transport mode plot
hover_text = []
# Iterate through all the rows and create the text we want to show when hovering over a certain data point. Add this to the column Text
for index, row in df.iterrows():
    hover_text.append(('Criticality: {criticality}<br>'+
                      'Vulnerability: {vulnerability}<br>' + 
                       'Road name: {roadname}<br>' + 
                       'Start LRP: {startLRP}<br>' + 
                       'End LRP: {endLRP}'
                      ).format(criticality=round(row['Criticality'],2),
                               vulnerability=round(row['Vulnerability'],2),
                               roadname=row['Road_Name'],
                               startLRP=row['Start LRP'],
                               endLRP=row['End LRP'])
                     )
df['Text'] = hover_text
data = []
# Get a list of all different modes of transport
modes = list(df)[12:28]
# Create the same number of traces, so we can switch on and off the different modes of transport
traces = ['trace' + str(i) for i in range(1,len(modes)+1)] 
# Define the traces with on the x-axis the Criticality and on the y-axis the Vulnerability 
# and make the size correspond to the density of that mode of transport
for trace, mode in zip(traces, modes):
    sizeref = 2.*max(df[mode])/(100**2)
    trace = go.Scatter(
        x=df['Criticality'], 
        y=df['Vulnerability'],
        mode='markers',
        name=mode,
        text=df['Text'],
        marker=dict(
            symbol='circle',
            sizemode='area',
            sizeref=sizeref,
            size=df[mode]/4,
            line=dict(
                width=2
            ),
        )
    )
    data.append(trace)
    
data = data
# Add the layout to the map. This includes the legend, title and the settings for the axes
layout = go.Layout(
    hovermode = 'closest',
    title='Criticality vs. vulnerability for all road segments per mode of transport',
    annotations=[
        dict(
            x=1.12,
            y=1.05,
            align="right",
            valign="top",
            text='Modes of transport',
            showarrow=False,
            xref="paper",
            yref="paper",
            xanchor="center",
            yanchor="top",
        )],
    xaxis=dict(
        title='Criticality',
        gridcolor='rgb(255, 255, 255)',
        range=[0,1],
        zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    ),
    yaxis=dict(
        title='Vulnerability',
        gridcolor='rgb(255, 255, 255)',
        range=[0,1],
        zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig,visible = 'legendonly')

In [4]:
#STEP 4 - CREATE SCATTERPLOT CRITICALITY/VULNERABIILTY/ROAD CATEGORIE BASED ON SCORES (TOP 10)
#make two traces, one for R roads one for N roads
trace_1_x = top_points_df[top_points_df["Road_cat"] == "R"]
trace_1_x = trace_1_x["segment_vulnerability_normalized"]
trace_1_x = trace_1_x.values.tolist()

trace_1_y = top_points_df[top_points_df["Road_cat"] == "R"]
trace_1_y = trace_1_y["Economic_Traffic_normalized"]
trace_1_y = trace_1_y.values.tolist()

trace_2_x = top_points_df[top_points_df["Road_cat"] == "N"]
trace_2_x = trace_2_x["segment_vulnerability_normalized"]
trace_2_x = trace_2_x.values.tolist()

trace_2_y = top_points_df[top_points_df["Road_cat"] == "N"]
trace_2_y = trace_2_y["Economic_Traffic_normalized"]
trace_2_y = trace_2_y.values.tolist()

#setup structure of traces
trace1 = {"x": trace_1_y,
          "y": trace_1_x, 
          "marker": {"color": porange, "size": 12}, 
          "mode": "markers", 
          "name": "R roads", 
          "type": "scatter"}

trace2 = {"x": trace_2_y, 
          "y": trace_2_x, 
          "marker": {"color": pblue, "size": 12}, 
          "mode": "markers", 
          "name": "N roads", 
          "type": "scatter", 
}

data = [trace1, trace2]
#setup layout of the plot
layout = {"title": "Top 10 road segments based on Criticalility and Vulnerability", 
          "xaxis": {"title": "Criticality (normalised)", }, 
          "yaxis": {"title": "Vulnerability (normalised)"}}

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [5]:
#Plot scatter plot of segment score
#Get list names and scores of each segment
list_of_road_segments = top_points_df.Link_No.tolist()
list_of_scores = top_points_df.segment_score.tolist()

#setup structure of plot
data = [go.Scatter(
            x=list_of_road_segments,
            y=list_of_scores,
            name='Absolute',
            mode = "markers",
            marker=dict(line=dict(color=porange, width=10)))]
#setup layout of plot (x axis and y axis)
layout = go.Layout(
            title='Segment score of top 10 road segments',
            yaxis=dict(title= 'Segment score', range = [0,0.2]),
            xaxis=dict(title= 'Road label'))
fig = dict(data=data, layout=layout)
py.iplot(fig)